This notebook is for examining the correlation between ML model predictions and the estimated assay yield

In [30]:
import pandas as pd
from tqdm import tqdm

from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem

import useful_rdkit_utils

from IPython.display import display

import plotly.express as px

Load spreadsheets

In [31]:
activity_df = pd.read_csv(
    '../data/raw/activity.csv', usecols=['SMILES y', 'mean_corr_activity']).rename(columns={'SMILES y': 'smiles', 'mean_corr_activity': 'activity'})

yield_df = pd.read_csv('../data/raw/yield.csv').rename(columns={'SMILES': 'smiles'})
# added column titles by hand - comment, yield and Plate Well instead of Well

display(activity_df)
display(yield_df)

,smiles,activity
0,O=C(CN(CC1C(Nc2cncc3ccccc23)=O)Cc(cc2)c1cc2Cl)...,51.8
1,O=C(CN(CC1C(Nc2cncc3ccccc23)=O)Cc(cc2)c1cc2Cl)...,9.9
2,O=C(CN(CC1C(Nc2cncc3ccccc23)=O)Cc(cc2)c1cc2Cl)...,9.5
3,[O-][N+](c(cc1)ccc1NC(CN(CC1C(Nc2cncc3ccccc23)...,83.6
4,CNC(c(cccc1)c1NC(CN(CC1C(Nc2cncc3ccccc23)=O)Cc...,71.1
...,...,...
295,N#Cc(cc1)cc(I)c1NC(CN(CC1C(Nc2cncc3ccccc23)=O)...,84.5
296,O=C(CN(CC1C(Nc2cncc3ccccc23)=O)Cc(cc2)c1cc2Cl)...,76.5
297,O=C(CN(CC1C(Nc2cncc3ccccc23)=O)Cc(cc2)c1cc2Cl)...,24.6
298,Cc1cccc(C)c1NC(CN(CC1)CCN1C(CN(CC1C(Nc2cncc3cc...,65.5


,Well,HOAt,Acid,P,random stuff,comment,yield,Plate Well,smiles
0,A1,36,35.0,0.0,29,NaN,0%,A1,O=C(CN(CC1C(Nc2cncc3ccccc23)=O)Cc(cc2)c1cc2Cl)...
1,A2,45,17.0,20.0,18,NaN,36%,A10,O=C(CN(CC1C(Nc2cncc3ccccc23)=O)Cc(cc2)c1cc2Cl)...
2,A3,45,42.0,0.0,13,NaN,0%,A11,N#Cc(ccc(NC(CN(CC1C(Nc2cncc3ccccc23)=O)Cc(cc2)...
3,A4,35,29.0,32.0,4,NaN,49%,A12,CCN(CC1)CCN1c(cc1)ccc1NC(CN(CC1C(Nc2cncc3ccccc...
4,A5,46,0.0,49.0,5,NaN,91%,A13,CCN(CC)S(c(cc1)ccc1NC(CN(CC1C(Nc2cncc3ccccc23)...
...,...,...,...,...,...,...,...,...,...
295,P14,40,36.0,0.0,24,NaN,0%,P5,COc1cccc(CCNC(CN(CC2C(Nc3cncc4ccccc34)=O)Cc(cc...
296,P15,42,38.0,1.0,19,amine,2%,P6,O=C(CN(CC1C(Nc2cncc3ccccc23)=O)Cc(cc2)c1cc2Cl)...
297,P16,27,2.0,0.0,71,amine,0%,P7,O=C(CN(CC1C(Nc2cncc3ccccc23)=O)Cc(cc2)c1cc2Cl)...
298,P17,56,0.0,41.0,3,NaN,93%,P8,O=C(CN(CC1C(Nc2cncc3ccccc23)=O)Cc(cc2)c1cc2Cl)...


Check SMILES merge

Merge dataframes and align plate well values

In [32]:
merged_df = activity_df.merge(yield_df, on='smiles')

df_with_yield = merged_df[['smiles', 'Plate Well', 'activity']]
for i, row in df_with_yield.iterrows():
    yield_slice = merged_df.query("Well == @row['Plate Well']").copy()
    df_with_yield.loc[i, 'yield (%)'] = yield_slice['yield'].str.rstrip('%').astype(float).values[0]

df_with_yield['Inhibition (%)'] = 100 - df_with_yield['activity']
display(df_with_yield)

/var/folders/f3/214cr8kd1_n76ygj86svh4nh0000gn/T/ipykernel_65489/2816453509.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



/var/folders/f3/214cr8kd1_n76ygj86svh4nh0000gn/T/ipykernel_65489/2816453509.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,smiles,Plate Well,activity,yield (%),Inhibition (%)
0,O=C(CN(CC1C(Nc2cncc3ccccc23)=O)Cc(cc2)c1cc2Cl)...,K19,51.8,96.0,48.2
1,O=C(CN(CC1C(Nc2cncc3ccccc23)=O)Cc(cc2)c1cc2Cl)...,J19,9.9,100.0,90.1
2,O=C(CN(CC1C(Nc2cncc3ccccc23)=O)Cc(cc2)c1cc2Cl)...,I19,9.5,93.0,90.5
3,[O-][N+](c(cc1)ccc1NC(CN(CC1C(Nc2cncc3ccccc23)...,L19,83.6,0.0,16.4
4,CNC(c(cccc1)c1NC(CN(CC1C(Nc2cncc3ccccc23)=O)Cc...,P11,71.1,0.0,28.9
...,...,...,...,...,...
295,N#Cc(cc1)cc(I)c1NC(CN(CC1C(Nc2cncc3ccccc23)=O)...,L11,84.5,0.0,15.5
296,O=C(CN(CC1C(Nc2cncc3ccccc23)=O)Cc(cc2)c1cc2Cl)...,P7,76.5,100.0,23.5
297,O=C(CN(CC1C(Nc2cncc3ccccc23)=O)Cc(cc2)c1cc2Cl)...,H10,24.6,98.0,75.4
298,Cc1cccc(C)c1NC(CN(CC1)CCN1C(CN(CC1C(Nc2cncc3cc...,H6,65.5,100.0,34.5


Visualise Yields

In [33]:
import time

px.scatter(df_with_yield,
            x='Inhibition (%)',
            y='yield (%)',
            width=1000,
            height=800,
            title='Measured Yield vs Measured Inhibition',
        labels={
            "yield (%)": "Measured Yield (%)",
            "Inhibition (%)": "Measured Inihibition (%)",
        },
            marginal_x='histogram',
            marginal_y='histogram',
            template='simple_white',)

In [35]:
from scipy.stats import spearmanr
print(f"Spearman correlation between inhibition and yield: {spearmanr(df_with_yield['Inhibition (%)'], df_with_yield['yield (%)'])[0]:.3f}")


Spearman correlation between inhibition and yield: 0.572


Load model predictions

In [43]:
rf_df = pd.read_csv('../data/predictions/rf_predictions.csv')
gp_df = pd.read_csv('../data/predictions/gp_predictions.csv')
ml_df = rf_df.merge(gp_df, on=['Molecule Name', 'SMILES'])
ml_df['mean_pred'] = ml_df[['RF Predicted Inhibition (%)', 'gp_pred']].mean(axis=1)
ml_df

,Molecule Name,SMILES,Measured Inhibition (%),RF Predicted Inhibition (%),index,Mean activity (%),Standard Deviation,Inhibition,amine_rbf_pred,amine_matern_pred,gp_pred,mean_pred
0,PCM-0223662,O=C(CN1Cc2ccc(Cl)cc2C(C(=O)Nc2cncc3ccccc23)C1)...,20.3,20.8332,0,79.7,0.3,20.3,14.775625,20.996022,17.885823,19.359512
1,PCM-0223661,O=C(CN1Cc2ccc(Cl)cc2C(C(=O)Nc2cncc3ccccc23)C1)...,20.9,20.1806,1,79.1,0.1,20.9,13.198525,20.127207,16.662866,18.421733
2,PCM-0223659,O=C(NC1CCN(C(=O)CN2Cc3ccc(Cl)cc3C(C(=O)Nc3cncc...,29.3,46.6270,3,70.7,0.3,29.3,33.456602,42.255552,37.856077,42.241538
3,PCM-0223658,COc1cccc(CCNC(=O)CN2Cc3ccc(Cl)cc3C(C(=O)Nc3cnc...,75.4,85.5704,4,24.6,0.5,75.4,76.218501,71.724706,73.971604,79.771002
4,PCM-0223657,Cc1nccn1-c1ccc(NC(=O)CN2Cc3ccc(Cl)cc3C(C(=O)Nc...,83.5,61.7364,5,16.5,0.3,83.5,43.946251,43.805654,43.875952,52.806176
...,...,...,...,...,...,...,...,...,...,...,...,...
290,PCM-0223367,CCN(CC)S(=O)(=O)c1ccc(NC(=O)CN2Cc3ccc(Cl)cc3C(...,13.0,58.6100,295,87.0,1.3,13.0,21.056010,28.420729,24.738369,41.674185
291,PCM-0223366,CCN1CCN(c2ccc(NC(=O)CN3Cc4ccc(Cl)cc4C(C(=O)Nc4...,97.4,63.3732,296,2.6,0.0,97.4,52.123949,48.901551,50.512750,56.942975
292,PCM-0223365,N#Cc1ccc(NC(=O)CN2Cc3ccc(Cl)cc3C(C(=O)Nc3cncc4...,17.4,31.6198,297,82.6,0.7,17.4,39.623642,37.806144,38.714893,35.167346
293,PCM-0223364,O=C(CN1Cc2ccc(Cl)cc2C(C(=O)Nc2cncc3ccccc23)C1)...,86.7,19.4728,298,13.3,0.2,86.7,19.741368,22.250994,20.996181,20.234490


Canonicalise SMILES and merge predictions with experimental measurements

In [45]:
def canonicalise_and_remove_stereo(smi):
    try:
        mol = Chem.MolFromSmiles(smi)
    except TypeError:
        print(f'{smi} is invalid, returning Nan')
        return None
    Chem.rdmolops.RemoveStereochemistry(mol)
    return Chem.MolToSmiles(mol)

ml_df['smiles'] = ml_df['SMILES'].apply(
    canonicalise_and_remove_stereo)

df_with_yield['smiles'] = df_with_yield['smiles'].apply(
    canonicalise_and_remove_stereo)

merged_df = df_with_yield.merge(ml_df, on='smiles')
merged_df

/var/folders/f3/214cr8kd1_n76ygj86svh4nh0000gn/T/ipykernel_65489/1285273804.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,smiles,Plate Well,activity,yield (%),Inhibition (%),Molecule Name,SMILES,Measured Inhibition (%),RF Predicted Inhibition (%),index,Mean activity (%),Standard Deviation,Inhibition,amine_rbf_pred,amine_matern_pred,gp_pred,mean_pred
0,O=C(Nc1cncc2ccccc12)C1CN(CC(=O)N2CCOCC2)Cc2ccc...,K19,51.8,96.0,48.2,PCM-0223563,O=C(Nc1cncc2ccccc12)C1CN(CC(=O)N2CCOCC2)Cc2ccc...,48.2,51.8978,99,51.8,0.5,48.2,67.405052,60.902176,64.153614,58.025707
1,O=C(CN1Cc2ccc(Cl)cc2C(C(=O)Nc2cncc3ccccc23)C1)...,J19,9.9,100.0,90.1,PCM-0223544,O=C(CN1Cc2ccc(Cl)cc2C(C(=O)Nc2cncc3ccccc23)C1)...,90.1,28.6458,118,9.9,0.1,90.1,45.529967,44.014798,44.772382,36.709091
2,O=C(CN1Cc2ccc(Cl)cc2C(C(=O)Nc2cncc3ccccc23)C1)...,I19,9.5,93.0,90.5,PCM-0223525,O=C(CN1Cc2ccc(Cl)cc2C(C(=O)Nc2cncc3ccccc23)C1)...,90.5,68.7526,137,9.5,0.4,90.5,104.069056,91.929904,97.999480,83.376040
3,O=C(CN1Cc2ccc(Cl)cc2C(C(=O)Nc2cncc3ccccc23)C1)...,L19,83.6,0.0,16.4,PCM-0223582,O=C(CN1Cc2ccc(Cl)cc2C(C(=O)Nc2cncc3ccccc23)C1)...,16.4,39.3330,80,83.6,0.6,16.4,61.633923,48.743634,55.188778,47.260889
4,CNC(=O)c1ccccc1NC(=O)CN1Cc2ccc(Cl)cc2C(C(=O)Nc...,P11,71.1,0.0,28.9,PCM-0223647,CNC(=O)c1ccccc1NC(=O)CN1Cc2ccc(Cl)cc2C(C(=O)Nc...,28.9,20.6552,15,71.1,0.5,28.9,42.575240,43.663796,43.119518,31.887359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,O=C(Nc1cncc2ccccc12)C1CN(CC(=O)N2CCN(c3ccc(Br)...,F3,75.7,87.0,24.3,PCM-0223470,O=C(Nc1cncc2ccccc12)C1CN(CC(=O)N2CCN(c3ccc(Br)...,24.3,23.0316,192,75.7,0.9,24.3,19.924737,27.465185,23.694961,23.363280
291,N#Cc1ccc(NC(=O)CN2Cc3ccc(Cl)cc3C(C(=O)Nc3cncc4...,L11,84.5,0.0,15.5,PCM-0223574,N#Cc1ccc(NC(=O)CN2Cc3ccc(Cl)cc3C(C(=O)Nc3cncc4...,15.5,21.6562,88,84.5,0.0,15.5,7.765679,22.544502,15.155091,18.405645
292,O=C(CN1Cc2ccc(Cl)cc2C(C(=O)Nc2cncc3ccccc23)C1)...,H10,24.6,98.0,75.4,PCM-0223497,O=C(CN1Cc2ccc(Cl)cc2C(C(=O)Nc2cncc3ccccc23)C1)...,75.4,41.1426,165,24.6,0.0,75.4,14.206393,21.256396,17.731394,29.436997
293,Cc1cccc(C)c1NC(=O)CN1CCN(C(=O)CN2Cc3ccc(Cl)cc3...,H6,65.5,100.0,34.5,PCM-0223511,Cc1cccc(C)c1NC(=O)CN1CCN(C(=O)CN2Cc3ccc(Cl)cc3...,34.5,28.2744,151,65.5,12.1,34.5,28.479242,35.603238,32.041240,30.157820


Plot predictions against yield

In [56]:
df_without_errors = merged_df.query('`Inhibition (%)` == `Measured Inhibition (%)`')

fig = px.scatter(df_without_errors, 
                 x='Inhibition (%)',
                 y='RF Predicted Inhibition (%)',
                 color='yield (%)',
                 hover_data=['Measured Inhibition (%)'],
                 width=1000,
                 height=800, 
                 title='Leave-One-Out Validation',
                 labels={
                     "RF Predicted Inhibition (%)": "ML Predicted Inhibition (%)",
                     "Inhibition (%)": "Measured Inihibition (%)",
                 },
                 template='simple_white',)

fig.add_shape(type='line',
              x0=0,
              y0=0,
              x1=100,
              y1=100,
              line=dict(color='black', dash='dash'),
              xref='x',
              yref='y',
              opacity=1,
              line_width=2
              )

fig.update_traces(marker={'size': 15})
fig.show()
